In [13]:
import pandas as pd

In [14]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\91814\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import numpy as np

In [16]:
df = pd.read_csv("C:/Users/91814/OneDrive/Desktop/Fake_Job_Detection/Job_posting_Fake_Detection.csv",engine="python")

In [17]:
df

,job_id,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,...,required_experience,required_education,industry,function,fraudulent,Country,State,City,min_salary,max_salalry
0,1,Marketing Intern,Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,none,0,1,0,...,Internship,none,none,Marketing,Real,US,NY,New York,none,none
1,2,Customer Service - Cloud Video Production,Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,...,Not Applicable,none,Marketing and Advertising,Customer Service,Real,NZ,,Auckland,none,none
2,3,Commissioning Machinery Assistant (CMA),none,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,none,0,1,0,...,none,none,none,none,Real,US,IA,Wever,none,none
3,4,Account Executive - Washington DC,Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,...,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,Real,US,DC,Washington,none,none
4,5,Bill Review Manager,none,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,...,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,Real,US,FL,Fort Worth,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,Sales,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,...,Mid-Senior level,none,Computer Software,Sales,Real,CA,ON,Toronto,none,none
17876,17877,Payroll Accountant,Accounting,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,...,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,Real,US,PA,Philadelphia,none,none
17877,17878,Project Cost Control Staff Engineer - Cost Con...,none,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,none,0,0,0,...,none,none,none,none,Real,US,TX,Houston,none,none
17878,17879,Graphic Designer,none,none,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,...,Not Applicable,Professional,Graphic Design,Design,Real,NG,LA,Lagos,none,none


In [18]:
df['fraudulent'] = df['fraudulent'].map({'Real': 0, 'Fake': 1})

In [19]:
df['fraudulent']

0        0
1        0
2        0
3        0
4        0
        ..
17875    0
17876    0
17877    0
17878    0
17879    0
Name: fraudulent, Length: 17880, dtype: int64

In [20]:
text_columns = ['title', 'company_profile', 'description', 'requirements', 'benefits']
df[text_columns] = df[text_columns].fillna('')

In [21]:
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

In [22]:
categorical_features = ['employment_type', 'required_experience', 'required_education', 'industry', 'function', 'Country', 'State', 'City']
df[categorical_features] = df[categorical_features].fillna('Unknown')

In [23]:
df[categorical_features].isnull().sum()

employment_type        0
required_experience    0
required_education     0
industry               0
function               0
Country                0
State                  0
City                   0
dtype: int64

In [24]:
for col in categorical_features:
    df[col] = LabelEncoder().fit_transform(df[col])

In [25]:
df['min_salary'] = pd.to_numeric(df['min_salary'], errors='coerce')
df['max_salalry'] = pd.to_numeric(df['max_salalry'], errors='coerce')
df[['min_salary', 'max_salalry']] = df[['min_salary', 'max_salalry']].fillna(df[['min_salary', 'max_salalry']].median())

In [26]:
df["has_company_logo"] = df["has_company_logo"].replace("Fake",1)

In [27]:
scaler = StandardScaler()

In [28]:
numeric_features = ['telecommuting', 'has_company_logo', 'has_questions', 'min_salary', 'max_salalry']
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [29]:
scaler.fit(df[['telecommuting', 'has_company_logo', 'has_questions', 'min_salary', 'max_salalry']])

StandardScaler()

In [30]:
vectorizer = TfidfVectorizer(max_features=5000)
X_text = vectorizer.fit_transform(df['combined_text'])

In [31]:
X_structured = df[numeric_features + categorical_features].values
X = np.hstack((X_structured, X_text.toarray()))
y = df['fraudulent']

In [32]:
# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [33]:
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

C:\Users\91814\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [35]:
def evaluate_model(model_name, y_true, y_pred):
    print(f"=== {model_name} ===")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))

evaluate_model("Logistic Regression", y_test, y_pred_lr)
evaluate_model("Random Forest", y_test, y_pred_rf)

=== Logistic Regression ===
Accuracy: 0.9544
Precision: 0.8571
Recall: 0.0694
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      3403
           1       0.86      0.07      0.13       173

    accuracy                           0.95      3576
   macro avg       0.91      0.53      0.55      3576
weighted avg       0.95      0.95      0.94      3576

=== Random Forest ===
Accuracy: 0.9801
Precision: 1.0000
Recall: 0.5896
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3403
           1       1.00      0.59      0.74       173

    accuracy                           0.98      3576
   macro avg       0.99      0.79      0.87      3576
weighted avg       0.98      0.98      0.98      3576

